In [1]:
import pandas as pd
import numpy as np
from math import sqrt, pi, log10
from PhysicalConstants import ReducedPlanckConstant as hbar
from PhysicalConstants import SpeedOfLight as c
from PhysicalConstants import VacuumPermittivity as eps0
from PhysicalConstants import ElectronCharge as qe

alpha_atomic_unit = 1.648_777_274e-41  # C^2.m^2.J^-1
d_atomic_unit = 8.478_353_625_5e-30

# Conversion functions

In [2]:
def log_gf_to_D(log_gf, delta_sigma):
    """
    Converts log(gf) from Ruczkowski to D in a.u. and sigma in cm-1
    """
    log_D = 1/2 * (log_gf + 8 - log10(303.76) -log10(delta_sigma))
    D = 10**log_D
    return D

def A_to_D(Ak, delta_sigma, Jk):
    """
    Converts A_J from Zhou et al. (https://arxiv.org/pdf/1002.0119.pdf)
    into D using formula (2) page 2
    """
    omega = c * 2 * pi * (delta_sigma * 1e2)
    D = sqrt(3 * pi * eps0 * hbar * c**3 * Ak * (2 * Jk + 1) / omega**3) / d_atomic_unit
    return D

def D_to_A(D, delta_sigma, Jk):
    """
    Inverses formula of A_to_D.
    """
    omega = c * 2 * pi * (delta_sigma * 1e2)
    Ak = (D * d_atomic_unit)**2 / (3 * pi * eps0 * hbar * c**3 * (2 * Jk + 1) / omega**3)
    return Ak

print('Example: for 1S0 to 3P1')
D_from_log = log_gf_to_D(-2.999, 14_504)
print(f'{D_from_log = :.3f} (a.u.)\n')
print('Example: for 1S0 to 1P1')
D_from_A = A_to_D(190.01*1e6, 21698.48, 1)
print(f'{D_from_A = :.3f} (a.u.)')

Example: for 1S0 to 3P1
D_from_log = 0.151 (a.u.)

Example: for 1S0 to 1P1
D_from_A = 5.248 (a.u.)


In [3]:
D_to_A(0.25, 38444.000-14504, 0)/2/np.pi/1e3

276.52816142724004

In [4]:
log_gf_to_D(-2.342, 38444.013 - 14504.334)

0.25013562327101624

In [5]:
A_to_D(4.216 * 1e6, 29303.449, 2)

0.6430189981157233

In [6]:
29303.449 + 14504

43807.449

In [7]:
# Transitions 3P1 vers 5s9d 3D1 et 3D2 d'après Boyd's thesis

omega = 5.52*1e15
Ak = 5.5*1e6

Ei = 14504  # energy of 3P1 in cm-1
Ek = Ei + omega/2/pi/c*1e-2


Jk = 1
D1 = sqrt(3 * pi * eps0 * hbar * c**3 * Ak * (2 * Jk + 1) / omega**3) / d_atomic_unit

Jk = 2
D2 = sqrt(3 * pi * eps0 * hbar * c**3 * Ak * (2 * Jk + 1) / omega**3) / d_atomic_unit

D1, D2, Ek

(0.5688540088065812, 0.7343873675104055, 43808.78277299632)

In [8]:
# Check conversion on 1S0 1P1 expected 5.25 a.u.

omega = 4.09*1e15
Jk = 1
Ak = 190.5*1e6
D = sqrt(3 * pi * eps0 * hbar * c**3 * Ak * (2 * Jk + 1) / omega**3) / d_atomic_unit

D

5.249169815435808

# Import data from .csv

In [9]:
tr = pd.read_csv('sr.csv', sep='\t')

# Adds the transition from 3P1 to 1S0
tr = tr.append(dict(zip(tr.columns, 
                        ['5s5p', '3P', '1', 14504,
                          '5s2', '1S', 0, 0, 0.151, 'NIST']
                        )), ignore_index=True)

tr = tr.astype({'J_i':str, 'J_k':str})

In [10]:
pd.options.display.max_rows = 999

display(tr)

,conf_i,term_i,J_i,E_i (cm-1),conf_k,term_k,J_k,E_k (cm-1),D (a.u.),source
0,5s2,1S,0,0,5s5p,3P,1,14504.000,0.151000,NIST
1,5s2,1S,0,0,5s5p,1P,1,21698.000,5.394400,NIST
2,5s2,1S,0,0,5s6p,1P,1,34098.000,0.266460,NIST
3,5s2,1S,0,0,5s6p,3P,1,33868.000,0.108000,Ruczkowski2016
4,5s2,1S,0,0,4d5p,3P,1,37302.000,0.043000,Ruczkowski2016
5,5s2,1S,0,0,5s7p,1P,1,38907.000,0.360560,NIST
6,5s2,1S,0,0,5s7p,3P,1,39426.000,0.053000,Ruczkowski2016
7,5s2,1S,0,0,4d5p,1P,1,41172.000,0.600000,NIST
8,5s2,1S,0,0,5s8p,3P,1,41720.000,0.033000,Ruczkowski2016
9,5s2,1S,0,0,5s8p,1P,1,42462.000,0.591610,NIST


## Add the computation of the wavelength in nm, and LS terms

In [11]:
tr.insert(tr.columns.get_loc('D (a.u.)'), 'wavelength (nm)', abs(round(1/(tr['E_k (cm-1)'] - tr['E_i (cm-1)'])*1e7, 2)))
tr.insert(tr.columns.get_loc('E_i (cm-1)'), 'Initial state', tr['conf_i'] + '-' + tr['term_i'] + tr['J_i'])
tr.insert(tr.columns.get_loc('E_k (cm-1)'), 'Final state', tr['conf_k'] + '-' + tr['term_k'] + tr['J_k']) 

In [12]:
tr.head()

,conf_i,term_i,J_i,Initial state,E_i (cm-1),conf_k,term_k,J_k,Final state,E_k (cm-1),wavelength (nm),D (a.u.),source
0,5s2,1S,0,5s2-1S0,0,5s5p,3P,1,5s5p-3P1,14504.0,689.46,0.15100,NIST
1,5s2,1S,0,5s2-1S0,0,5s5p,1P,1,5s5p-1P1,21698.0,460.87,5.39440,NIST
2,5s2,1S,0,5s2-1S0,0,5s6p,1P,1,5s6p-1P1,34098.0,293.27,0.26646,NIST
3,5s2,1S,0,5s2-1S0,0,5s6p,3P,1,5s6p-3P1,33868.0,295.26,0.10800,Ruczkowski2016
4,5s2,1S,0,5s2-1S0,0,4d5p,3P,1,4d5p-3P1,37302.0,268.08,0.04300,Ruczkowski2016


# Table from 1S0

## Filter the data

In [13]:
tr_g = tr.query('conf_i == "5s2" & term_i == "1S" & J_i == "0"')
display(tr_g)

,conf_i,term_i,J_i,Initial state,E_i (cm-1),conf_k,term_k,J_k,Final state,E_k (cm-1),wavelength (nm),D (a.u.),source
0,5s2,1S,0,5s2-1S0,0,5s5p,3P,1,5s5p-3P1,14504.0,689.46,0.151000,NIST
1,5s2,1S,0,5s2-1S0,0,5s5p,1P,1,5s5p-1P1,21698.0,460.87,5.394400,NIST
2,5s2,1S,0,5s2-1S0,0,5s6p,1P,1,5s6p-1P1,34098.0,293.27,0.266460,NIST
3,5s2,1S,0,5s2-1S0,0,5s6p,3P,1,5s6p-3P1,33868.0,295.26,0.108000,Ruczkowski2016
4,5s2,1S,0,5s2-1S0,0,4d5p,3P,1,4d5p-3P1,37302.0,268.08,0.043000,Ruczkowski2016
5,5s2,1S,0,5s2-1S0,0,5s7p,1P,1,5s7p-1P1,38907.0,257.02,0.360560,NIST
6,5s2,1S,0,5s2-1S0,0,5s7p,3P,1,5s7p-3P1,39426.0,253.64,0.053000,Ruczkowski2016
7,5s2,1S,0,5s2-1S0,0,4d5p,1P,1,4d5p-1P1,41172.0,242.88,0.600000,NIST
8,5s2,1S,0,5s2-1S0,0,5s8p,3P,1,5s8p-3P1,41720.0,239.69,0.033000,Ruczkowski2016
9,5s2,1S,0,5s2-1S0,0,5s8p,1P,1,5s8p-1P1,42462.0,235.50,0.591610,NIST


In [14]:
col_drop = ['Initial state', 'conf_i', 'term_i', 'J_i', 'E_i (cm-1)', 'conf_k', 'term_k', 'J_k','E_k (cm-1)']
tr_g_drop = tr_g.drop(columns=col_drop)
tr_g_drop

,Final state,wavelength (nm),D (a.u.),source
0,5s5p-3P1,689.46,0.151000,NIST
1,5s5p-1P1,460.87,5.394400,NIST
2,5s6p-1P1,293.27,0.266460,NIST
3,5s6p-3P1,295.26,0.108000,Ruczkowski2016
4,4d5p-3P1,268.08,0.043000,Ruczkowski2016
5,5s7p-1P1,257.02,0.360560,NIST
6,5s7p-3P1,253.64,0.053000,Ruczkowski2016
7,4d5p-1P1,242.88,0.600000,NIST
8,5s8p-3P1,239.69,0.033000,Ruczkowski2016
9,5s8p-1P1,235.50,0.591610,NIST


## Load polarizability data

In [15]:
J = 0

alpha_g = pd.read_csv('tables_1064_nm/alphas_5s2-1S0.csv', sep=';', decimal=',')
alpha_g = alpha_g.loc[:, ~alpha_g.columns.str.contains('^Unnamed')]

alpha_g.insert(loc=4, column='alpha_s (a.u.)', value=alpha_g['alpha_k'] / sqrt(3 * (2 * J + 1)) / alpha_atomic_unit)
alpha_g.insert(loc=5, column='weight (%)', value=round(alpha_g['alpha_s (a.u.)'] / alpha_g['alpha_s (a.u.)'].sum() * 100, 2))
alpha_g.sort_values('alpha_s (a.u.)', ascending=False)
alpha_g.drop(columns=['Initial', 'alpha_k', 'K'], inplace=True)
alpha_g.rename(columns={'Final':'Final state'}, inplace=True)
alpha_g.sort_values('alpha_s (a.u.)', ascending=False, inplace=True)
alpha_g

,Final state,alpha_s (a.u.),weight (%)
1,5s5p-1P1,241.546050,97.65
8,4d5p-1P1,1.349694,0.55
10,5s8p-1P1,1.268175,0.51
12,5s9p-1P1,0.740645,0.30
6,5s7p-1P1,0.519199,0.21
14,5s10p-1P1,0.418767,0.17
0,5s5p-3P1,0.396510,0.16
2,5s6p-1P1,0.329719,0.13
16,5s11p-1P1,0.217536,0.09
17,5s12p-1P1,0.137068,0.06


## Join the tables

In [16]:
df1 = tr_g_drop.copy()
df1 = df1.join(alpha_g.set_index('Final state'), on='Final state').sort_values('alpha_s (a.u.)', ascending=False)

In [17]:
df1 = df1[['Final state', 'wavelength (nm)', 'D (a.u.)',  'alpha_s (a.u.)',
       'weight (%)', 'source']]
df1
print(df1['alpha_s (a.u.)'].sum())

247.3144433082969


In [18]:
# df1.to_latex('polarizabilities_1S0.tex', index=False)

# Table from 3P1

In [19]:
tr_3P1 = tr.query('conf_i == "5s5p" & term_i == "3P" & J_i == "1"')
display(tr_3P1)

,conf_i,term_i,J_i,Initial state,E_i (cm-1),conf_k,term_k,J_k,Final state,E_k (cm-1),wavelength (nm),D (a.u.),source
30,5s5p,3P,1,5s5p-3P1,14504,5s4d,3D,1,5s4d-3D1,18159.000,2735.98,2.32200,Cooper2018
31,5s5p,3P,1,5s5p-3P1,14504,5s4d,3D,2,5s4d-3D2,18218.000,2692.51,4.01900,Cooper2018
32,5s5p,3P,1,5s5p-3P1,14504,5s4d,1D,2,5s4d-1D2,20149.000,1771.48,0.19000,Cooper2018
33,5s5p,3P,1,5s5p-3P1,14504,5s6d,3D,1,5s6d-3D1,39685.552,397.12,0.98635,Zhou2010
34,5s5p,3P,1,5s5p-3P1,14504,5s6d,3D,2,5s6d-3D2,39690.488,397.04,1.70840,Zhou2010
35,5s5p,3P,1,5s5p-3P1,14504,5s8s,3S,1,5s8s-3S1,40761.089,380.85,0.47880,Zhou2010
36,5s5p,3P,1,5s5p-3P1,14504,5s7d,3D,1,5s7d-3D1,41864.049,365.50,0.66100,Zhou2010
37,5s5p,3P,1,5s5p-3P1,14504,5s7d,3D,2,5s7d-3D2,41868.969,365.43,1.14480,Zhou2010
38,5s5p,3P,1,5s5p-3P1,14504,5s9s,3S,1,5s9s-3S1,42450.849,357.82,0.32350,Zhou2010
39,5s5p,3P,1,5s5p-3P1,14504,5s8d,3D,1,5s8d-3D1,43066.349,350.11,0.47950,Zhou2010


In [20]:
col_drop = ['Initial state', 'conf_i', 'term_i', 'J_i', 'E_i (cm-1)', 'conf_k', 'term_k', 'J_k','E_k (cm-1)']
tr_3P1_drop = tr_3P1.drop(columns=col_drop)
display(tr_3P1_drop)

,Final state,wavelength (nm),D (a.u.),source
30,5s4d-3D1,2735.98,2.32200,Cooper2018
31,5s4d-3D2,2692.51,4.01900,Cooper2018
32,5s4d-1D2,1771.48,0.19000,Cooper2018
33,5s6d-3D1,397.12,0.98635,Zhou2010
34,5s6d-3D2,397.04,1.70840,Zhou2010
35,5s8s-3S1,380.85,0.47880,Zhou2010
36,5s7d-3D1,365.50,0.66100,Zhou2010
37,5s7d-3D2,365.43,1.14480,Zhou2010
38,5s9s-3S1,357.82,0.32350,Zhou2010
39,5s8d-3D1,350.11,0.47950,Zhou2010


In [21]:
J = 1

alpha_3P1 = pd.read_csv('tables_1064_nm/alphas_5s5p-3P1_test.csv', sep=';',decimal=',')
alpha_3P1 = alpha_3P1.loc[:, ~alpha_3P1.columns.str.contains('^Unnamed')]


len(alpha_3P1.columns)

alpha_3P1.insert(loc=len(alpha_3P1.columns), column='alpha_s', value=alpha_3P1['alpha_0'] / sqrt(3 * (2 * J + 1)) / alpha_atomic_unit)
alpha_3P1.insert(loc=len(alpha_3P1.columns), column='weight_s (%)', value=round(alpha_3P1['alpha_s'] / alpha_3P1['alpha_s'].abs().sum() * 100, 2))

alpha_3P1.insert(loc=len(alpha_3P1.columns), column='alpha_v', value=(alpha_3P1['alpha_1']
                                               * (- sqrt((2 * J) / ((J + 1) * (2 * J + 1)))) 
                                               / alpha_atomic_unit
                                              )
              )
alpha_3P1.insert(loc=len(alpha_3P1.columns), column='weight_v (%)', value=round(alpha_3P1['alpha_v'] / alpha_3P1['alpha_v'].abs().sum() * 100, 2))

alpha_3P1.insert(loc=len(alpha_3P1.columns), column='alpha_t', value=(alpha_3P1['alpha_2']
                                                * (- sqrt((2 * J * (2 * J - 1))))
                                                / (3 * (J + 1) * (2 * J + 1) * (2 * J + 3))
                                                / alpha_atomic_unit
                                               )
              )
alpha_3P1.insert(loc=len(alpha_3P1.columns), column='weight_t (%)', value=round(alpha_3P1['alpha_t'] / alpha_3P1['alpha_t'].abs().sum() * 100, 2))

alpha_3P1.sort_values(by='alpha_s', key=abs, inplace=True, ascending=False)
alpha_3P1.drop(columns=['Initial', 'alpha_0', 'alpha_1', 'alpha_2'], inplace=True)
alpha_3P1.rename(columns={'Final':'Final state'}, inplace=True)
display(alpha_3P1)
print(alpha_3P1['alpha_s'].sum())

alpha_3P1['Final state'][0]='5s2-1S0'
display(alpha_3P1)

,Final state,alpha_s,weight_s (%),alpha_v,weight_v (%),alpha_t,weight_t (%)
15,5s6s-3S1,67.656139,26.38,-65.625358,-17.74,3.565792e+00,35.15
19,5s5d-3D2,40.583799,15.82,27.884785,7.54,-4.277908e-01,-4.22
2,5s4d-3D2,-39.252968,-15.31,-148.997931,-40.27,4.137626e-01,4.08
22,5p2-3P2,23.552447,9.18,15.684288,4.24,-2.482646e-01,-2.45
20,5p2-3P0,20.969844,8.18,-28.578230,-7.72,-2.210416e+00,-21.79
21,5p2-3P1,16.324038,6.37,-11.013214,-2.98,8.603524e-01,8.48
1,5s4d-3D1,-12.819780,-5.00,49.447328,13.36,-6.756617e-01,-6.66
18,5s5d-3D1,12.155062,4.74,-8.357750,-2.26,6.406280e-01,6.32
29,4d2-3P2,8.724413,3.40,4.069164,1.10,-9.196339e-02,-0.91
28,4d2-3P1,5.313165,2.07,-2.489155,-0.67,2.800284e-01,2.76


151.70342115095713


C:\Users\Romaric\AppData\Local\Temp\ipykernel_8524\3270799604.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alpha_3P1['Final state'][0]='5s2-1S0'


,Final state,alpha_s,weight_s (%),alpha_v,weight_v (%),alpha_t,weight_t (%)
15,5s6s-3S1,67.656139,26.38,-65.625358,-17.74,3.565792e+00,35.15
19,5s5d-3D2,40.583799,15.82,27.884785,7.54,-4.277908e-01,-4.22
2,5s4d-3D2,-39.252968,-15.31,-148.997931,-40.27,4.137626e-01,4.08
22,5p2-3P2,23.552447,9.18,15.684288,4.24,-2.482646e-01,-2.45
20,5p2-3P0,20.969844,8.18,-28.578230,-7.72,-2.210416e+00,-21.79
21,5p2-3P1,16.324038,6.37,-11.013214,-2.98,8.603524e-01,8.48
1,5s4d-3D1,-12.819780,-5.00,49.447328,13.36,-6.756617e-01,-6.66
18,5s5d-3D1,12.155062,4.74,-8.357750,-2.26,6.406280e-01,6.32
29,4d2-3P2,8.724413,3.40,4.069164,1.10,-9.196339e-02,-0.91
28,4d2-3P1,5.313165,2.07,-2.489155,-0.67,2.800284e-01,2.76


## Join the tables

In [22]:
df = tr_3P1_drop.copy()
df = df.join(alpha_3P1.set_index('Final state'), on='Final state').sort_values(by='alpha_s', key=abs, ascending=False)

display(df)

,Final state,wavelength (nm),D (a.u.),source,alpha_s,weight_s (%),alpha_v,weight_v (%),alpha_t,weight_t (%)
44,5s6s-3S1,688.04,3.42500,Cooper2018,67.656139,26.38,-65.625358,-17.74,3.565792e+00,35.15
48,5s5d-3D2,487.38,3.67300,Cooper2018,40.583799,15.82,27.884785,7.54,-4.277908e-01,-4.22
31,5s4d-3D2,2692.51,4.01900,Cooper2018,-39.252968,-15.31,-148.997931,-40.27,4.137626e-01,4.08
51,5p2-3P2,472.37,2.86500,Cooper2018,23.552447,9.18,15.684288,4.24,-2.482646e-01,-2.45
49,5p2-3P0,483.35,2.65700,Cooper2018,20.969844,8.18,-28.578230,-7.72,-2.210416e+00,-21.79
50,5p2-3P1,478.56,2.36200,Cooper2018,16.324038,6.37,-11.013214,-2.98,8.603524e-01,8.48
30,5s4d-3D1,2735.98,2.32200,Cooper2018,-12.819780,-5.00,49.447328,13.36,-6.756617e-01,-6.66
47,5s5d-3D1,487.73,2.00900,Cooper2018,12.155062,4.74,-8.357750,-2.26,6.406280e-01,6.32
58,4d2-3P2,330.84,2.21000,Ruczkowski2016,8.724413,3.40,4.069164,1.10,-9.196339e-02,-0.91
57,4d2-3P1,332.31,1.72000,Ruczkowski2016,5.313165,2.07,-2.489155,-0.67,2.800284e-01,2.76


In [23]:
df = df[['Final state', 'wavelength (nm)', 'D (a.u.)',  'alpha_s', 'alpha_v', 'alpha_t', 'source']]
display(df)

,Final state,wavelength (nm),D (a.u.),alpha_s,alpha_v,alpha_t,source
44,5s6s-3S1,688.04,3.42500,67.656139,-65.625358,3.565792e+00,Cooper2018
48,5s5d-3D2,487.38,3.67300,40.583799,27.884785,-4.277908e-01,Cooper2018
31,5s4d-3D2,2692.51,4.01900,-39.252968,-148.997931,4.137626e-01,Cooper2018
51,5p2-3P2,472.37,2.86500,23.552447,15.684288,-2.482646e-01,Cooper2018
49,5p2-3P0,483.35,2.65700,20.969844,-28.578230,-2.210416e+00,Cooper2018
50,5p2-3P1,478.56,2.36200,16.324038,-11.013214,8.603524e-01,Cooper2018
30,5s4d-3D1,2735.98,2.32200,-12.819780,49.447328,-6.756617e-01,Cooper2018
47,5s5d-3D1,487.73,2.00900,12.155062,-8.357750,6.406280e-01,Cooper2018
58,4d2-3P2,330.84,2.21000,8.724413,4.069164,-9.196339e-02,Ruczkowski2016
57,4d2-3P1,332.31,1.72000,5.313165,-2.489155,2.800284e-01,Ruczkowski2016


In [30]:
df.to_latex('polarizabilities_3P1.tex', index=False)

None


# Look for a specific transition

In [25]:
tr = pd.read_csv('sr.csv', sep='\t')
tr_search = tr.query('(conf_i == "5s2" or conf_k == "5s2") and source == "Ruczkowski2016"')
display(tr_search)

,conf_i,term_i,J_i,E_i (cm-1),conf_k,term_k,J_k,E_k (cm-1),D (a.u.),source
3,5s2,1S,0,0,5s6p,3P,1,33868.0,0.108,Ruczkowski2016
4,5s2,1S,0,0,4d5p,3P,1,37302.0,0.043,Ruczkowski2016
6,5s2,1S,0,0,5s7p,3P,1,39426.0,0.053,Ruczkowski2016
8,5s2,1S,0,0,5s8p,3P,1,41720.0,0.033,Ruczkowski2016
10,5s2,1S,0,0,5s9p,3P,1,42990.0,0.017,Ruczkowski2016
12,5s2,1S,0,0,5s10p,3P,1,43761.0,0.012,Ruczkowski2016
14,5s2,1S,0,0,5s11p,3P,1,44265.0,0.009,Ruczkowski2016


In [26]:
tr = pd.read_csv('sr.csv', sep='\t')
tr_search = tr.query('conf_i == "5s5p"')
display(tr_search)

,conf_i,term_i,J_i,E_i (cm-1),conf_k,term_k,J_k,E_k (cm-1),D (a.u.),source
25,5s5p,3P,0,14318,5s6s,3S,1,29039.000,2.02480,NIST
26,5s5p,3P,0,14318,5s5d,3D,1,35007.000,2.34520,NIST
27,5s5p,3P,0,14318,5p2,3P,1,35400.000,2.49000,NIST
28,5s5p,3P,0,14318,5s7s,3S,1,37425.000,0.60828,NIST
29,5s5p,3P,0,14318,4d2,3P,1,44596.000,1.76070,NIST
30,5s5p,3P,1,14504,5s4d,3D,1,18159.000,2.32200,Cooper2018
31,5s5p,3P,1,14504,5s4d,3D,2,18218.000,4.01900,Cooper2018
32,5s5p,3P,1,14504,5s4d,1D,2,20149.000,0.19000,Cooper2018
33,5s5p,3P,1,14504,5s6d,3D,1,39685.552,0.98635,Zhou2010
34,5s5p,3P,1,14504,5s6d,3D,2,39690.488,1.70840,Zhou2010


In [27]:
tr = pd.read_csv('sr.csv', sep='\t')
tr_search = tr.query('(conf_i == "5s5p" and term_i == "3P" and J_i == 1)'
                    'or (conf_k == "5s5p" and term_k == "3P" and J_k == 1)')
display(tr_search)
tr_search.describe()

,conf_i,term_i,J_i,E_i (cm-1),conf_k,term_k,J_k,E_k (cm-1),D (a.u.),source
0,5s2,1S,0,0,5s5p,3P,1,14504.000,0.15100,NIST
30,5s5p,3P,1,14504,5s4d,3D,1,18159.000,2.32200,Cooper2018
31,5s5p,3P,1,14504,5s4d,3D,2,18218.000,4.01900,Cooper2018
32,5s5p,3P,1,14504,5s4d,1D,2,20149.000,0.19000,Cooper2018
33,5s5p,3P,1,14504,5s6d,3D,1,39685.552,0.98635,Zhou2010
34,5s5p,3P,1,14504,5s6d,3D,2,39690.488,1.70840,Zhou2010
35,5s5p,3P,1,14504,5s8s,3S,1,40761.089,0.47880,Zhou2010
36,5s5p,3P,1,14504,5s7d,3D,1,41864.049,0.66100,Zhou2010
37,5s5p,3P,1,14504,5s7d,3D,2,41868.969,1.14480,Zhou2010
38,5s5p,3P,1,14504,5s9s,3S,1,42450.849,0.32350,Zhou2010


,J_i,E_i (cm-1),J_k,E_k (cm-1),D (a.u.)
count,30.000000,30.000000,30.000000,30.000000,30.000000
mean,0.966667,14020.533333,1.200000,36300.748367,1.298205
std,0.182574,2648.055991,0.714384,8505.381890,1.179816
min,0.000000,0.000000,0.000000,14504.000000,0.045000
25%,1.000000,14504.000000,1.000000,35010.750000,0.299125
50%,1.000000,14504.000000,1.000000,37934.000000,0.875800
75%,1.000000,14504.000000,2.000000,42912.474000,2.159750
max,1.000000,14504.000000,2.000000,44730.000000,4.019000


In [28]:
c= 3e8
2*3.14159*c/4.09e15

4.608689486552567e-07